In [1]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=657015cd4cd9ef5a6bc9f88289ccb72ef5f0b50d34f6590686e31770b53c77fd
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [2]:
!pip install datasets loralib sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [3]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [4]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00


In [5]:
import os
import torch
import json
import random
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from bert_score import score
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import numpy as np


In [6]:
from huggingface_hub import notebook_login

notebook_login()

Prompt tuning

In [7]:
# Load LLaMA 2 model
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto", token=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [8]:
print(model.hf_device_map)

{'': 0}


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

In [10]:
import torch

print("Using device:", model.device)  # Check where the model is loaded
print("CUDA available:", torch.cuda.is_available())  # Should be True
print("CUDA device count:", torch.cuda.device_count())  # Should be >0


Using device: cuda:0
CUDA available: True
CUDA device count: 1


In [14]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForPreTraining

#tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
bert_model = AutoModelForPreTraining.from_pretrained("nlpaueb/legal-bert-base-uncased")

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

#Concatenate

In [11]:
# Load JSON file
file_path = "/content/drive/MyDrive/AI4BH/CourseWork/SNLP/contractnli.json"
def load_json(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

data = load_json(file_path)

In [12]:
# Extract queries and retrieved chunks for all entries
def extract_all_data(data):
    data_samples = []
    for item in data:  # Iterate directly over the list
        query = item["query"]
        retrieved_chunks = [chunk["text"] for chunk in item.get("retrieved_chunks_unranked", [])[:3]]  # Get up to 3 texts
        data_samples.append({"retrieved_chunks": retrieved_chunks, "user_query": query})
    return data_samples

In [13]:
# Extract all queries
all_data = extract_all_data(data)

In [14]:
def generate_prompt(retrieved_chunks, user_query):
    formatted_chunks = "\n\n".join(retrieved_chunks) if retrieved_chunks else "No relevant excerpts available."
    return f"""### Instruction:
You are an AI assistant specializing in legal contract analysis. Your task is to carefully examine the **provided NDA excerpts** and **answer the user's question accurately**.

Follow these guidelines:
Read the NDA clause carefully. Identify any terms, conditions, or restrictions related to the user's question.
Answer explicitly based on the clause. If the NDA clearly states the information being asked, explain it clearly and accurately.
Do not ignore relevant details. If the clause contains conditions, restrictions, or exceptions, **mention them in your answer.
If the NDA does not provide a direct answer, say so. Do not assume or infer information that is not stated.
Support your answer with key phrases from the NDA clause when necessary.
Avoid unnecessary repetition or legal jargon. The goal is to make the answer **clear and understandable.

### Retrieved chunks:
{formatted_chunks}

### User's Question:
{user_query}

### Answer:
"""

In [15]:
# Function to generate model responses
def generate_responses(sampled_data):
    responses = []
    for sample in sampled_data:
        prompt = generate_prompt(sample["retrieved_chunks"], sample["user_query"])
        inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                max_length=1000,
                temperature=0.3,
                top_p=0.9,
                num_return_sequences=1,
                do_sample=True,
                attention_mask=inputs.attention_mask
            )

        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        if "### Answer" in response:
            response = response.split("### Answer")[-1].strip()  # Remove prompt parts
        elif "Answer:" in response:
            response = response.split("Answer:")[-1].strip()  # Alternative case

        responses.append(response)
    return responses

'''

        Remove everything before "Answer:" to keep only the actual response
        if "Answer:" in response:
            response = response.split("Answer:")[-1].strip()

        responses.append(response)

    return responses'''

'\n\n        Remove everything before "Answer:" to keep only the actual response\n        if "Answer:" in response:\n            response = response.split("Answer:")[-1].strip()\n\n        responses.append(response)\n\n    return responses'

In [16]:
# Function to compute BERTScore using LegalBERT
def compute_bertscore(retrieved_chunks, model_responses):
    references = [" ".join(chunks).lower() for chunks in retrieved_chunks]
    model_responses = [text.lower() for text in model_responses]

    P, R, F1 = score(model_responses, references, model_type="nlpaueb/legal-bert-base-uncased", num_layers=12)
    return list(F1.numpy())

# Function to compute ROUGE-Recall
def compute_rouge_recall(retrieved_chunks, model_responses):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    recall_scores = []

    for ref_chunks, pred in zip(retrieved_chunks, model_responses):
        ref_text = " ".join(ref_chunks)
        scores = scorer.score(ref_text, pred)

        # Compute the average recall score instead of using just ROUGE-1
        avg_recall = (scores["rouge1"].recall + scores["rouge2"].recall + scores["rougeL"].recall) / 3
        recall_scores.append(avg_recall)

    return recall_scores

In [17]:
tokenizer.pad_token = tokenizer.eos_token

In [18]:
# Generate responses using fine-tuned LLaMA 2
model_responses = generate_responses(all_data)

# Store results in a new JSON file
output_data = []
for sample, response in zip(all_data, model_responses):
    output_data.append({
        "query": sample["user_query"],
        "retrieved_chunks": sample["retrieved_chunks"],
        "generated_response": response
    })

# Save to a new JSON file
output_file_path = "Llama 3 k3.json"
with open(output_file_path, "w") as f:
    json.dump(output_data, f, indent=4)

print(f"\n **All responses saved to {output_file_path}**")

# Compute factual consistency scores using LegalBERT BERTScore
bert_scores = compute_bertscore(
    retrieved_chunks=[sample["retrieved_chunks"] for sample in all_data],
    model_responses=model_responses
)

# Compute ROUGE-Recall scores
rouge_recall_scores = compute_rouge_recall(
    retrieved_chunks=[sample["retrieved_chunks"] for sample in all_data],
    model_responses=model_responses
)

# Print each query, response, and all evaluation scores
for i, (sample, response, bert_score, rouge_score) in enumerate(
        zip(all_data, model_responses, bert_scores, rouge_recall_scores)):

    print(f"\n **Query {i+1}:** {sample['user_query']}")
    print(f" **Retrieved Chunks:**")
    for chunk in sample["retrieved_chunks"]:
        print(f"   - {chunk}")
    print(f" **Model Response:** {response}")
    print(f" **BERTScore-F1 (LegalBERT):** {bert_score:.4f}")
    print(f" **ROUGE-Recall:** {rouge_score:.4f}")
    print("=" * 100)

# Compute and print average scores
avg_bert_score = np.mean(bert_scores)
avg_rouge_recall = np.mean(rouge_recall_scores)

print(f"\n **Average BERTScore-F1 (LegalBERT):** {avg_bert_score:.4f}")
print(f" **Average ROUGE-Recall:** {avg_rouge_recall:.4f}")


 **All responses saved to Llama 3 k3.json**


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


 **Query 1:** Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
 **Retrieved Chunks:**
   - 4 Definition of Confidential Information
“Confidential Information” means any Idea disclosed to Mentor, all data and information, know-how, business concepts, software, procedures, products, services, development projects, and programmes contained in such Idea and/or its description and any conclusions. Confidential Information does not include:
> information already known or independently developed by Mentor prior to the disclosure of any Idea;
   - 3 Payment ..........................................................................................................................................4
4 Definition of confidential information ..............................................................................................4
5 Non-disclosure of 

#First Chunk


In [11]:
# Load JSON file
file_path = "/content/drive/MyDrive/AI4BH/CourseWork/SNLP/contractnli.json"  # Ensure the file is uploaded in Google Colab
with open(file_path, "r") as f:
    data = json.load(f)

# Extract queries and take ONLY the FIRST retrieved chunk
def extract_first_chunk(data):
    data_samples = []
    for item in data:
        query = item["query"]
        retrieved_chunks = item.get("retrieved_chunks_unranked", [])

        # Take ONLY the FIRST retrieved chunk (if available)
        first_chunk = retrieved_chunks[0]["text"] if retrieved_chunks else "No relevant excerpts available."

        data_samples.append({"retrieved_chunk": first_chunk, "user_query": query})

    return data_samples

In [12]:
# Generate structured prompt
def generate_prompt(retrieved_chunk, user_query):
    return f"""### Instruction:
You are an AI assistant specializing in legal contract analysis. Your task is to carefully examine the **provided Retrieved Chunk** and **answer the user's question accurately**.

Follow these guidelines:
Read the clause carefully. Identify any terms, conditions, or restrictions related to the user's question.
Answer explicitly based on the clause. If the clause clearly states the information being asked, explain it clearly and accurately.
Do not ignore relevant details. If the clause contains conditions, restrictions, or exceptions, **mention them in your answer.
If the clause does not provide a direct answer, say so. Do not assume or infer information that is not stated.
Support your answer with key phrases from the clause clause when necessary.
Avoid unnecessary repetition or legal jargon. The goal is to make the answer **clear and understandable.

### Retrieved Chunk:
{retrieved_chunk}

### User's Question:
{user_query}

### Answer:
"""

In [13]:
# Generate model responses
def generate_responses(all_data):
    responses = []
    for sample in all_data:
        prompt = generate_prompt(sample["retrieved_chunk"], sample["user_query"])
        inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                max_length=1000,
                temperature=0.3,
                top_p=0.90,
                num_return_sequences=1,
                do_sample=True,
                attention_mask=inputs.attention_mask
            )

        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        if "### Answer" in response:
            response = response.split("### Answer")[-1].strip()  # Remove prompt parts
        elif "Answer:" in response:
            response = response.split("Answer:")[-1].strip()  # Alternative case

        responses.append(response)
    return responses

In [14]:
# Compute BERTScore (LegalBERT)
def compute_bertscore(retrieved_chunks, model_responses):
    references = [chunk.lower() for chunk in retrieved_chunks]  # Use first retrieved chunk
    model_responses = [text.lower() for text in model_responses]

    P, R, F1 = score(model_responses, references, model_type="nlpaueb/legal-bert-base-uncased", num_layers=12)
    return list(F1.numpy())

# Compute ROUGE-Recall
def compute_rouge_recall(retrieved_chunks, model_responses):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    recall_scores = []
    for ref_chunk, pred in zip(retrieved_chunks, model_responses):
        scores = scorer.score(ref_chunk, pred)
        avg_recall = (scores["rouge1"].recall + scores["rouge2"].recall + scores["rougeL"].recall) / 3
        recall_scores.append(avg_recall)
    return recall_scores

In [15]:
tokenizer.pad_token = tokenizer.eos_token

In [16]:
# Process Data (Now using the FIRST retrieved chunk)
all_data = extract_first_chunk(data)
model_responses = generate_responses(all_data)

# Save results to JSON file (Google Colab)
output_data = []
for sample, response in zip(all_data, model_responses):
    output_data.append({
        "query": sample["user_query"],
        "retrieved_chunk": sample["retrieved_chunk"],
        "generated_response": response
    })

output_file_path = "generated_responses_first_chunk.json"
with open(output_file_path, "w") as f:
    json.dump(output_data, f, indent=4)

print(f"\n✅ **All responses saved to {output_file_path}**")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for


✅ **All responses saved to generated_responses_first_chunk.json**


In [17]:
# Compute Evaluation Metrics
bert_scores = compute_bertscore([sample["retrieved_chunk"] for sample in all_data], model_responses)
rouge_recall_scores = compute_rouge_recall([sample["retrieved_chunk"] for sample in all_data], model_responses)

# Print individual results
for i, (query, retrieved, response, bert_score, rouge_score) in enumerate(
        zip([sample["user_query"] for sample in all_data],
            [sample["retrieved_chunk"] for sample in all_data],
            model_responses, bert_scores, rouge_recall_scores)):

    print(f"\n🔹 **Query {i+1}:** {query}")
    print(f"🔹 **Retrieved Chunk:** {retrieved}")
    print(f"🔹 **Generated Response:** {response}")
    print(f"🔹 **BERTScore-F1 (LegalBERT):** {bert_score:.4f}")
    print(f"🔹 **ROUGE-Recall:** {rouge_score:.4f}")
    print("=" * 100)

# Compute and print average scores
avg_bert_score = np.mean(bert_scores)
avg_rouge_recall = np.mean(rouge_recall_scores)

print(f"\n🔹 **Average BERTScore-F1 (LegalBERT):** {avg_bert_score:.4f}")
print(f"🔹 **Average ROUGE-Recall:** {avg_rouge_recall:.4f}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]


🔹 **Query 1:** Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
🔹 **Retrieved Chunk:** 4 Definition of Confidential Information
“Confidential Information” means any Idea disclosed to Mentor, all data and information, know-how, business concepts, software, procedures, products, services, development projects, and programmes contained in such Idea and/or its description and any conclusions. Confidential Information does not include:
> information already known or independently developed by Mentor prior to the disclosure of any Idea;
🔹 **Generated Response:** :
According to the Non-Disclosure Agreement, the term "Confidential Information" refers to any Idea disclosed to Mentor, as well as various types of data and information, including know-how, business concepts, software, procedures, products, services, development projects, and programmes. 

# Best chunks

In [22]:
from sentence_transformers import SentenceTransformer, util

# Load the Sentence-BERT model
similarity_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
# Load JSON file
file_path = "/content/drive/MyDrive/AI4BH/CourseWork/SNLP/contractnli.json"
def load_json(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

data = load_json(file_path)

In [24]:
def extract_best_chunk(data):
    data_samples = []
    for item in data:
        query = item["query"]
        retrieved_chunks = item.get("retrieved_chunks_unranked", [])

        if not retrieved_chunks:  # If no retrieved chunks, return placeholder
            best_chunk = "No relevant excerpts available."
        else:
            # Compute similarity scores
            query_embedding = similarity_model.encode(query, convert_to_tensor=True)
            chunk_embeddings = [similarity_model.encode(chunk["text"], convert_to_tensor=True) for chunk in retrieved_chunks]
            similarity_scores = [util.pytorch_cos_sim(query_embedding, chunk_embedding).item() for chunk_embedding in chunk_embeddings]

            # Select the chunk with the highest similarity score
            best_chunk_index = similarity_scores.index(max(similarity_scores))
            best_chunk = retrieved_chunks[best_chunk_index]["text"]

        data_samples.append({"retrieved_chunk": best_chunk, "user_query": query})

    return data_samples

In [25]:
def generate_prompt(retrieved_chunks, user_query):
    formatted_chunks = "\n\n".join(retrieved_chunks) if retrieved_chunks else "No relevant excerpts available."
    return f"""### Instruction:
You are an AI assistant specializing in legal contract analysis. Your task is to carefully examine the **provided NDA excerpts** and **answer the user's question accurately**.

Follow these guidelines:
Read the NDA clause carefully. Identify any terms, conditions, or restrictions related to the user's question.
Answer explicitly based on the clause. If the NDA clearly states the information being asked, explain it clearly and accurately.
Do not ignore relevant details. If the clause contains conditions, restrictions, or exceptions, **mention them in your answer.
If the NDA does not provide a direct answer, say so. Do not assume or infer information that is not stated.
Support your answer with key phrases from the NDA clause when necessary.
Avoid unnecessary repetition or legal jargon. The goal is to make the answer **clear and understandable.

### Retrieved chunks:
{formatted_chunks}

### User's Question:
{user_query}

### Answer:
"""

In [31]:
# Generate model responses
def generate_responses(all_data):
    responses = []
    for sample in all_data:
        prompt = generate_prompt(sample["retrieved_chunk"], sample["user_query"])
        inputs = tokenizer(prompt, return_tensors="pt", padding=True).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                max_length=2000,
                temperature=0.3,
                top_p=0.9,
                num_return_sequences=1,
                do_sample=True,
                attention_mask=inputs.attention_mask
            )

        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        if "### Answer" in response:
            response = response.split("### Answer")[-1].strip()  # Remove prompt parts
        elif "Answer:" in response:
            response = response.split("Answer:")[-1].strip()  # Alternative case

        responses.append(response)
    return responses

In [32]:
# Compute BERTScore (LegalBERT)
def compute_bertscore(retrieved_chunks, model_responses):
    references = [chunk.lower() for chunk in retrieved_chunks]  # Use best retrieved chunk
    model_responses = [text.lower() for text in model_responses]

    P, R, F1 = score(model_responses, references, model_type="nlpaueb/legal-bert-base-uncased", num_layers=12)
    return list(F1.numpy())

# Compute ROUGE-Recall
def compute_rouge_recall(retrieved_chunks, model_responses):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    recall_scores = []
    for ref_chunk, pred in zip(retrieved_chunks, model_responses):
        scores = scorer.score(ref_chunk, pred)
        avg_recall = (scores["rouge1"].recall + scores["rouge2"].recall + scores["rougeL"].recall) / 3
        recall_scores.append(avg_recall)
    return recall_scores

In [16]:
all_data = extract_best_chunk(data)

NameError: name 'extract_best_chunk' is not defined

In [34]:
model_responses = generate_responses(all_data)

# Save results to JSON file (for Google Colab)
output_data = []
for sample, response in zip(all_data, model_responses):
    output_data.append({
        "query": sample["user_query"],
        "retrieved_chunk": sample["retrieved_chunk"],
        "generated_response": response
    })

output_file_path = "generated_responses.json"
with open(output_file_path, "w") as f:
    json.dump(output_data, f, indent=4)

print(f"\n **All responses saved to generated_responses.json**")

# Evaluate the model
bert_scores = compute_bertscore(
    retrieved_chunks=[sample["retrieved_chunk"] for sample in all_data],
    model_responses=model_responses
)
rouge_recall_scores = compute_rouge_recall(
    retrieved_chunks=[sample["retrieved_chunk"] for sample in all_data],
    model_responses=model_responses
)

# Print results for sanity check
for i, (sample, response, bert_score, rouge_score) in enumerate(
        zip(all_data, model_responses, bert_scores, rouge_recall_scores)):

    print(f"\n **Query {i+1}:** {sample['user_query']}")
    print(f" **Best Retrieved Chunk:** {sample['retrieved_chunk']}")
    print(f" **Model Response:** {response}")
    print(f" **BERTScore-F1 (LegalBERT):** {bert_score:.4f}")
    print(f" **ROUGE-Recall:** {rouge_score:.4f}")
    print("=" * 100)

# Print Average Scores
avg_bert_score = np.mean(bert_scores)
avg_rouge_recall = np.mean(rouge_recall_scores)

print(f"\n **Average BERTScore-F1 (LegalBERT):** {avg_bert_score:.4f}")
print(f" **Average ROUGE-Recall:** {avg_rouge_recall:.4f}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for


 **All responses saved to generated_responses.json**

 **Query 1:** Consider the Non-Disclosure Agreement between CopAcc and ToP Mentors; Does the document indicate that the Agreement does not grant the Receiving Party any rights to the Confidential Information?
 **Best Retrieved Chunk:** Mentor shall notify Organiser immediately in writing of any misuse or misappropriation of any Confidential Information that may come to Mentor’s attention. Mentor agrees to segregate all Confidential Information relating to this agreement from Confidential Information of others to avoid commingling.
6 Discontinuation of Use, Return of Materials
At Organiser’s first request, Mentor shall:
(a) discontinue all use of Confidential Information;
 **Model Response:** The Non-Disclosure Agreement between CopAcc and ToP Mentors does not grant the Receiving Party any rights to the Confidential Information. The Agreement states that the Receiving Party shall not disclose, copy, or use any Confidential Informati

#Null


In [29]:
# Generate responses using fine-tuned LLaMA 2
model_responses = generate_responses(all_data)

# Store results in a new JSON file
output_data = []
for sample, response in zip(all_data, model_responses):
    output_data.append({
        "query": sample["user_query"],
        "retrieved_chunks": sample["retrieved_chunks"],
        "generated_response": response
    })

# Save to a new JSON file
output_file_path = "generated_responses.json"
with open(output_file_path, "w") as f:
    json.dump(output_data, f, indent=4)

print(f"\n **All responses saved to {output_file_path}**")

# Compute factual consistency scores using LegalBERT BERTScore
bert_scores = compute_bertscore(
    retrieved_chunks=[sample["retrieved_chunks"] for sample in all_data],
    model_responses=model_responses
)

# Compute ROUGE-Recall scores
rouge_recall_scores = compute_rouge_recall(
    retrieved_chunks=[sample["retrieved_chunks"] for sample in all_data],
    model_responses=model_responses
)

# Print each query, response, and all evaluation scores
for i, (sample, response, bert_score, rouge_score) in enumerate(
        zip(all_data, model_responses, bert_scores, rouge_recall_scores)):

    print(f"\n **Query {i+1}:** {sample['user_query']}")
    print(f" **Retrieved Chunks:**")
    for chunk in sample["retrieved_chunks"]:
        print(f"   - {chunk}")
    print(f" **Model Response:** {response}")
    print(f" **BERTScore-F1 (LegalBERT):** {bert_score:.4f}")
    print(f" **ROUGE-Recall:** {rouge_score:.4f}")
    print("=" * 100)

# Compute and print average scores
avg_bert_score = np.mean(bert_scores)
avg_rouge_recall = np.mean(rouge_recall_scores)

print(f"\n **Average BERTScore-F1 (LegalBERT):** {avg_bert_score:.4f}")
print(f" **Average ROUGE-Recall:** {avg_rouge_recall:.4f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-d24d4672de9e>", line 2, in <cell line: 0>
    model_responses = generate_responses(all_data)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-26-64e91e9861c2>", line 9, in generate_responses
    output_ids = model.generate(
                 ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py", line 2223, in generate
    result = self._sample(
             ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py", line 3200, in _sample
    while self._has_unfinished_sequences(
KeyboardInterrupt

TypeError: object of type 'NoneType' has no len()

In [16]:
def compute_rouge_recall(retrieved_chunks, model_responses):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    recall_scores = []

    for ref, pred in zip(retrieved_chunks, model_responses):
        scores = scorer.score(ref, pred)

        # Compute the average recall score instead of using just ROUGE-1
        avg_recall = (scores["rouge1"].recall + scores["rouge2"].recall + scores["rougeL"].recall) / 3
        recall_scores.append(avg_recall)

    return recall_scores

In [38]:
# Load your JSON file
file_path = "/content/drive/MyDrive/AI4BH/CourseWork/SNLP/contractnli.json"
data = load_json(file_path)

# Extract 100 samples
sampled_data = extract_data(data, sample_size=100)

# Generate responses using fine-tuned LLaMA 2
model_responses = generate_responses(sampled_data)

# Compute factual consistency scores using LegalBERT BERTScore
bert_scores = compute_bertscore(
    retrieved_chunks=[sample["retrieved_chunks"] for sample in sampled_data],
    model_responses=model_responses
)

# Compute ROUGE-Recall scores
rouge_recall_scores = compute_rouge_recall(
    retrieved_chunks=[sample["retrieved_chunks"] for sample in sampled_data],
    model_responses=model_responses
)

# Print each query, response, and all evaluation scores
for i, (sample, response, bert_score, rouge_score) in enumerate(
        zip(sampled_data, model_responses, bert_scores, rouge_recall_scores)):

    print(f"\n **Query {i+1}:** {sample['user_query']}")
    print(f" **Retrieved Chunks:**")
    for chunk in sample["retrieved_chunks"]:
        print(f"   - {chunk}")
    print(f" **Model Response:** {response}")
    print(f" **BERTScore-F1 (LegalBERT):** {bert_score:.4f}")
    print(f" **ROUGE-Recall:** {rouge_score:.4f}")
    print("=" * 100)

# Compute and print average scores
avg_bert_score = np.mean(bert_scores)
avg_rouge_recall = np.mean(rouge_recall_scores)

print(f"\n🔹 **Average BERTScore-F1 (LegalBERT):** {avg_bert_score:.4f}")
print(f"🔹 **Average ROUGE-Recall:** {avg_rouge_recall:.4f}")


 **Query 1:** Consider EFCA's Non-Disclosure Agreement; Does the document include a clause that prevents the Receiving Party from disclosing the fact that the Agreement was agreed upon or negotiated?
 **Retrieved Chunks:**
   - EFCA agrees to keep the existence and nature of this Agreement confidential and not to use the same or the name of the disclosing party (or of any company in the Group of Companies of which the disclosing party forms part) in any publicity, advertisement or other disclosure with regard to this Agreement without the prior written consent of the disclosing party.
 **Model Response:** Based on the provided NDA excerpt, the answer to your question is **no**. The clause does not include any restrictions on disclosing the fact that the Agreement was agreed upon or negotiated. The clause only pertains to the confidentiality of the Agreement's existence and nature, and does not address the disclosure of information about the Agreement's existence or the parties involve

In [17]:
# Function to compute BERTScore using LegalBERT
def compute_bertscore(retrieved_chunks, model_responses):
    references = [" ".join(chunks).lower() for chunks in retrieved_chunks]
    model_responses = [text.lower() for text in model_responses]

    P, R, F1 = score(model_responses, references, model_type="nlpaueb/legal-bert-base-uncased", num_layers=12)
    return list(F1.numpy())

# Function to compute ROUGE-Recall
def compute_rouge_recall(retrieved_chunks, model_responses):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    recall_scores = []

    for ref_chunks, pred in zip(retrieved_chunks, model_responses):
        ref_text = " ".join(ref_chunks)
        scores = scorer.score(ref_text, pred)

        # Compute the average recall score instead of using just ROUGE-1
        avg_recall = (scores["rouge1"].recall + scores["rouge2"].recall + scores["rougeL"].recall) / 3
        recall_scores.append(avg_recall)

    return recall_scores

In [ ]:
prompt = "User Query: Consider the Non-Disclosure Agreement between ON Semiconductor and Industry Analysts; Does the document restrict the use of Confidential Information to the purposes stated in the Agreement?\nRetrieved Chunks: Confidential Information disclosed during the ON Semiconductor Industry Analyst Conference to be held by invitation only on April 18-20, 2011, in Scottsdale, Arizona, hereunder may only be used for the following purpose: For evaluation and discussion of market trends and data refinement in the areas of ON Semiconductor's focus market segments.\nAnswer:"
# Tokenize and check length amd response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
print("Tokenized Input:", inputs["input_ids"].shape)
output_ids = model.generate(**inputs, max_length=300, temperature=0.7, top_p=0.9)

# Decode the response
response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(response)

Tokenized Input: torch.Size([1, 143])
User Query: Consider the Non-Disclosure Agreement between ON Semiconductor and Industry Analysts; Does the document restrict the use of Confidential Information to the purposes stated in the Agreement?
Retrieved Chunks: Confidential Information disclosed during the ON Semiconductor Industry Analyst Conference to be held by invitation only on April 18-20, 2011, in Scottsdale, Arizona, hereunder may only be used for the following purpose: For evaluation and discussion of market trends and data refinement in the areas of ON Semiconductor's focus market segments.
Answer: Yes, the Non-Disclosure Agreement between ON Semiconductor and Industry Analysts restricts the use of Confidential Information to the purposes stated in the Agreement. The document specifically states that the Confidential Information disclosed during the conference may only be used for the purpose of evaluating and discussing market trends and data refinement in the areas of ON Semico

In [ ]:
output_ids = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    num_return_sequences=2,
    max_new_tokens=200
)

for i, out in enumerate(output_ids):
    response = tokenizer.decode(out, skip_special_tokens=True)
    print(f"Response {i+1}: {response}")

Response 1: User Query: Consider the Non-Disclosure Agreement between ON Semiconductor and Industry Analysts; Does the document restrict the use of Confidential Information to the purposes stated in the Agreement?
Retrieved Chunks: Confidential Information disclosed during the ON Semiconductor Industry Analyst Conference to be held by invitation only on April 18-20, 2011, in Scottsdale, Arizona, hereunder may only be used for the following purpose: For evaluation and discussion of market trends and data refinement in the areas of ON Semiconductor's focus market segments.
Answer: According to the Non-Disclosure Agreement between ON Semiconductor and industry analysts, the document restricts the use of Confidential Information to the purposes stated in the Agreement. The Confidential Information disclosed during the conference may only be used for evaluating and discussing market trends and data refinement in the areas of ON Semiconductor's focus market segments. The agreement does not a

In [ ]:
# Function to calculate Perplexity
def calculate_perplexity(model, tokenizer, text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt").to(device)
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()


# Compute perplexity for the generated response
perplexity_score = calculate_perplexity(model, tokenizer, response)
print(f"Perplexity Score: {perplexity_score}")

Perplexity Score: 4.362564563751221


In [ ]:
#Prompt Structure
retrieved_chunk = "Confidential Information disclosed during the ON Semiconductor Industry Analyst Conference to be held by invitation only on April 18-20, 2011, in Scottsdale, Arizona, hereunder may only be used for the following purpose: For evaluation and discussion of market trends and data refinement in the areas of ON Semiconductor's focus market segments."
user_query = "Consider the Non-Disclosure Agreement between ON Semiconductor and Industry Analysts; Does the document restrict the use of Confidential Information to the purposes stated in the Agreement?"

prompt = f"""### Instruction:
Analyze the following Non-Disclosure Agreement (NDA) and determine if it restricts the use of Confidential Information to the purposes stated in the Agreement.

### NDA Clause:
"{retrieved_chunk}"

### Question:
{user_query}

### Answer:
"""

In [ ]:
# Tokenize and generate response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
print("Tokenized Input:", inputs["input_ids"].shape)

output_ids = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9
)

response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Response:\n", response)

Tokenized Input: torch.Size([1, 200])
Generated Response:
 ### Instruction:
Analyze the following Non-Disclosure Agreement (NDA) and determine if it restricts the use of Confidential Information to the purposes stated in the Agreement.

### NDA Clause:
"Confidential Information disclosed during the ON Semiconductor Industry Analyst Conference to be held by invitation only on April 18-20, 2011, in Scottsdale, Arizona, hereunder may only be used for the following purpose: For evaluation and discussion of market trends and data refinement in the areas of ON Semiconductor's focus market segments."

### Question:
Consider the Non-Disclosure Agreement between ON Semiconductor and Industry Analysts; Does the document restrict the use of Confidential Information to the purposes stated in the Agreement?

### Answer:
Yes, the Non-Disclosure Agreement restricts the use of Confidential Information to the purposes stated in the Agreement. The clause explicitly states that the Confidential Informati

# *Evaluation*




Perplexity

In [ ]:
# Function to calculate Perplexity
def calculate_perplexity(model, tokenizer, text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt").to(device)
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()


# Compute perplexity for the generated response
perplexity_score = calculate_perplexity(model, tokenizer, response)
print(f"Perplexity Score: {perplexity_score}")

Perplexity Score: 3.1478211879730225


In [ ]:
import numpy as np

In [ ]:
import json
import random
from bert_score import score

with open("/content/drive/MyDrive/SNLP/contractnli.json", "r") as f:
    data = json.load(f)


In [ ]:
# Extract all query-answer pairs
data_samples = []
for item in data["tests"]:
    query = item["query"]
    retrieved_chunks = " ".join([snippet["answer"] for snippet in item["snippets"]])  # Combine multiple snippets
    data_samples.append({"retrieved_chunk": retrieved_chunks, "user_query": query})

In [ ]:
# Randomly sample 100 queries
random.seed(42)  # For reproducibility
sampled_data = random.sample(data_samples, min(15, len(data_samples)))  # Take 100 or total available if less

def generate_prompt(retrieved_chunk, user_query):
    return f"""### Instruction:
You are an AI assistant specializing in legal contract analysis. Your goal is to **carefully examine the provided NDA clause** and answer the user's question **accurately and concisely**.

Follow these guidelines:
Read the NDA clause carefully. Identify any terms, conditions, or restrictions related to the user's question.
Answer explicitly based on the clause. If the NDA clearly states the information being asked, explain it clearly and accurately.
Do not ignore relevant details. If the clause contains conditions, restrictions, or exceptions, **mention them in your answer.
If the NDA does not provide a direct answer, say so. Do not assume or infer information that is not stated.
Support your answer with key phrases from the NDA clause when necessary.
Avoid unnecessary repetition or legal jargon. The goal is to make the answer **clear and understandable.

### NDA Clause:
"{retrieved_chunk}"

### User's Question:
{user_query}

### Answer:
"""

In [ ]:
# Print 5 sample prompts to inspect input
print("\n🔹 **Sample Prompts:**\n")
for i in range(5):
    sample = sampled_data[i]
    prompt = generate_prompt(sample["retrieved_chunk"], sample["user_query"])
    print(f"🔹 **Prompt {i+1}**:\n{prompt}\n{'='*80}\n")


🔹 **Sample Prompts:**

🔹 **Prompt 1**:
### Instruction:
You are an AI assistant specializing in legal contract analysis. Your goal is to **carefully examine the provided NDA clause** and answer the user's question **accurately and concisely**.  

Follow these guidelines:  
Read the NDA clause carefully. Identify any terms, conditions, or restrictions related to the user's question.  
Answer explicitly based on the clause. If the NDA clearly states the information being asked, explain it clearly and accurately.  
Do not ignore relevant details. If the clause contains conditions, restrictions, or exceptions, **mention them in your answer.  
If the NDA does not provide a direct answer, say so. Do not assume or infer information that is not stated.  
Support your answer with key phrases from the NDA clause when necessary.  
Avoid unnecessary repetition or legal jargon. The goal is to make the answer **clear and understandable. 

### NDA Clause:
"Confidential Information includes, without 

In [ ]:
def calculate_perplexity(model, tokenizer, text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt").to(device)
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

# Compute perplexity for each structured prompt
perplexity_scores = []
model_responses = []

for sample in sampled_data:
    prompt = generate_prompt(sample["retrieved_chunk"], sample["user_query"])
    perplexity = calculate_perplexity(model, tokenizer, prompt)
    perplexity_scores.append(perplexity)
    # Generate model response
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(**inputs, max_length=500, temperature=0.7, top_p=0.9)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    model_responses.append((perplexity, prompt, response))
    print(f"Perplexity for Prompt: {perplexity:.4f}")

# Compute average perplexity
average_perplexity = np.mean(perplexity_scores)
print(f"\n **Average Perplexity Over {len(sampled_data)} Prompts: {average_perplexity:.4f}**")

Perplexity for Prompt: 11.5202
Perplexity for Prompt: 14.1540
Perplexity for Prompt: 10.8835
Perplexity for Prompt: 10.6112
Perplexity for Prompt: 10.8184
Perplexity for Prompt: 9.7159
Perplexity for Prompt: 7.5475
Perplexity for Prompt: 10.9745
Perplexity for Prompt: 10.3874
Perplexity for Prompt: 11.7669
Perplexity for Prompt: 10.3031
Perplexity for Prompt: 10.6371
Perplexity for Prompt: 7.6991
Perplexity for Prompt: 12.2755
Perplexity for Prompt: 8.5433

 **Average Perplexity Over 15 Prompts: 10.5225**


In [ ]:
# Print 10 model responses with their perplexity scores
print("\n🔹 **Model Responses with Perplexity Scores:**\n")
top_10_responses = sorted(model_responses, key=lambda x: x[0])[:15]  # Get 10 lowest perplexity responses

for i, (perplexity, prompt, response) in enumerate(top_10_responses):
    print(f"🔹 **Response {i+1} (Perplexity: {perplexity:.4f})**\n")
    print(f"**Prompt:**\n{prompt}\n")
    print(f"**Model Response:**\n{response}\n")
    print("="*100 + "\n")


🔹 **Model Responses with Perplexity Scores:**

🔹 **Response 1 (Perplexity: 7.5475)**

**Prompt:**
### Instruction:
You are an AI assistant specializing in legal contract analysis. Your goal is to **carefully examine the provided NDA clause** and answer the user's question **accurately and concisely**.  

Follow these guidelines:  
Read the NDA clause carefully. Identify any terms, conditions, or restrictions related to the user's question.  
Answer explicitly based on the clause. If the NDA clearly states the information being asked, explain it clearly and accurately.  
Do not ignore relevant details. If the clause contains conditions, restrictions, or exceptions, **mention them in your answer.  
If the NDA does not provide a direct answer, say so. Do not assume or infer information that is not stated.  
Support your answer with key phrases from the NDA clause when necessary.  
Avoid unnecessary repetition or legal jargon. The goal is to make the answer **clear and understandable. 

#

Bert Score


In [ ]:
# Load JSON file
def load_json(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

# Extract queries and retrieved chunks
def extract_data(data, sample_size=100):
    data_samples = []
    for item in data["tests"]:
        query = item["query"]
        retrieved_chunks = " ".join([snippet["answer"] for snippet in item["snippets"]])  # Combine multiple snippets
        data_samples.append({"retrieved_chunk": retrieved_chunks, "user_query": query})

    # Take a random sample (or full dataset if smaller than sample_size)
    return random.sample(data_samples, min(sample_size, len(data_samples)))

In [ ]:
# Function to generate structured prompt
def generate_prompt(retrieved_chunk, user_query):
    return f"""### Instruction:
You are an AI assistant specializing in legal contract analysis. Your goal is to **carefully examine the provided NDA clause** and answer the user's question **accurately and concisely**.

Follow these guidelines:
- **Read the NDA clause carefully.** Identify any terms, conditions, or restrictions related to the user's question.
- **Answer explicitly based on the clause.** If the NDA clearly states the information being asked, explain it **clearly and accurately**.
- **Do not ignore relevant details.** If the clause contains conditions, restrictions, or exceptions, **mention them** in your answer.
- **If the NDA does not provide a direct answer, say so.** Do not assume or infer information that is not stated.
- **Support your answer with key phrases** from the NDA clause when necessary.
- **Avoid unnecessary repetition or legal jargon.** The goal is to make the answer **clear and understandable**.

### NDA Clause:
"{retrieved_chunk}"

### User's Question:
{user_query}

### Answer:
"""

In [ ]:
# Function to generate model responses
def generate_responses(sampled_data):
    responses = []
    for sample in sampled_data:
        prompt = generate_prompt(sample["retrieved_chunk"], sample["user_query"])
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                max_length=300,
                temperature=0.7,
                top_p=0.9
            )

        response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        responses.append(response)

    return responses


In [ ]:

# Function to compute BERTScore for factual consistency
def compute_bertscore(retrieved_chunks, model_responses):
    P, R, F1 = score(model_responses, retrieved_chunks, lang="en", rescale_with_baseline=True)
    return F1.mean().item()  # Return average F1-score

# Load your JSON file (Replace with actual path)
file_path = "/content/drive/MyDrive/SNLP/contractnli.json"
data = load_json(file_path)

# Extract 100 samples
sampled_data = extract_data(data, sample_size=100)

# Generate model responses
model_responses = generate_responses(sampled_data)

# Compute factual consistency score using BERTScore
factual_consistency_score = compute_bertscore(
    retrieved_chunks=[sample["retrieved_chunk"] for sample in sampled_data],
    model_responses=model_responses
)

# Print the factual consistency score
print(f"🔹 **Factual Consistency Score (BERTScore-F1): {factual_consistency_score:.4f}**")

In [ ]:
# Function to compute BERTScore for factual consistency
def compute_bertscore(retrieved_chunks, model_responses):
    P, R, F1 = score(model_responses, retrieved_chunks, lang="en", rescale_with_baseline=True)
    return F1.mean().item()  # Return average F1-score

# Load your JSON file (Replace with actual path)
file_path = "/content/drive/MyDrive/SNLP/contractnli.json"
data = load_json(file_path)

# Extract 100 samples
sampled_data = extract_data(data, sample_size=100)

# Replace this with actual model responses (for now using retrieved_chunks as dummy responses)
model_responses = [sample["retrieved_chunk"] for sample in sampled_data]

# Compute factual consistency score using BERTScore
factual_consistency_score = compute_bertscore(
    retrieved_chunks=[sample["retrieved_chunk"] for sample in sampled_data],
    model_responses=model_responses
)

# Print the factual consistency score
print(f"🔹 **Factual Consistency Score (BERTScore-F1): {factual_consistency_score:.4f}**")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🔹 **Factual Consistency Score (BERTScore-F1): 1.0000**


LoRA

In [ ]:
# Load LLaMA 2 model
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto", token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [ ]:
from peft import LoraConfig, PeftModel, get_peft_model

In [ ]:
# LoRA Configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_feat

In [ ]:
#Prompt Structure
retrieved_chunk = "Confidential Information disclosed during the ON Semiconductor Industry Analyst Conference to be held by invitation only on April 18-20, 2011, in Scottsdale, Arizona, hereunder may only be used for the following purpose: For evaluation and discussion of market trends and data refinement in the areas of ON Semiconductor's focus market segments."

user_query = "Consider the Non-Disclosure Agreement between ON Semiconductor and Industry Analysts; Does the document restrict the use of Confidential Information to the purposes stated in the Agreement?"

prompt = f"""### Instruction:
Analyze the following Non-Disclosure Agreement (NDA) and determine if it restricts the use of Confidential Information to the purposes stated in the Agreement.

### NDA Clause:
"{retrieved_chunk}"

### Question:
{user_query}

### Answer:
"""

In [ ]:
# Tokenize and generate response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
print("Tokenized Input:", inputs["input_ids"].shape)

output_ids = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9
)

response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Generated Response:\n", response)

Tokenized Input: torch.Size([1, 200])
Generated Response:
 ### Instruction:
Analyze the following Non-Disclosure Agreement (NDA) and determine if it restricts the use of Confidential Information to the purposes stated in the Agreement.

### NDA Clause:
"Confidential Information disclosed during the ON Semiconductor Industry Analyst Conference to be held by invitation only on April 18-20, 2011, in Scottsdale, Arizona, hereunder may only be used for the following purpose: For evaluation and discussion of market trends and data refinement in the areas of ON Semiconductor's focus market segments."

### Question:
Consider the Non-Disclosure Agreement between ON Semiconductor and Industry Analysts; Does the document restrict the use of Confidential Information to the purposes stated in the Agreement?

### Answer:
Yes, the Non-Disclosure Agreement restricts the use of Confidential Information to the purposes stated in the Agreement. The Agreement specifically states that the Confidential Info

In [ ]:
# Function to calculate Perplexity
def calculate_perplexity(model, tokenizer, text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt").to(device)
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()


# Compute perplexity for the generated response
perplexity_score = calculate_perplexity(model, tokenizer, response)
print(f"Perplexity Score: {perplexity_score}")

Perplexity Score: 3.9033308029174805


In [ ]:
# Randomly sample 100 queries
random.seed(42)  # For reproducibility
sampled_data = random.sample(data_samples, min(100, len(data_samples)))  # Take 100 or total available if less

# Function to generate structured prompt
def generate_prompt(retrieved_chunk, user_query):
    return f"""### Instruction:
Analyze the following Non-Disclosure Agreement (NDA) and determine if it restricts the use of Confidential Information to the purposes stated in the Agreement.

### NDA Clause:
"{retrieved_chunk}"

### Question:
{user_query}

### Answer:
"""

In [ ]:
def calculate_perplexity(model, tokenizer, text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt").to(device)
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

# Compute perplexity for each structured prompt
perplexity_scores = []
for sample in sampled_data:
    prompt = generate_prompt(sample["retrieved_chunk"], sample["user_query"])
    perplexity = calculate_perplexity(model, tokenizer, prompt)
    perplexity_scores.append(perplexity)
    print(f"Perplexity for Prompt: {perplexity:.4f}")

# Compute average perplexity
average_perplexity = np.mean(perplexity_scores)
print(f"\n🔹 **Average Perplexity Over {len(sampled_data)} Prompts: {average_perplexity:.4f}**")

Perplexity for Prompt: 13.4433
Perplexity for Prompt: 18.4549
Perplexity for Prompt: 11.7361
Perplexity for Prompt: 10.6326
Perplexity for Prompt: 13.0195
Perplexity for Prompt: 8.4840
Perplexity for Prompt: 6.4269
Perplexity for Prompt: 12.2462
Perplexity for Prompt: 8.9534
Perplexity for Prompt: 14.1942
Perplexity for Prompt: 11.1442
Perplexity for Prompt: 10.8058
Perplexity for Prompt: 6.6392
Perplexity for Prompt: 15.7714
Perplexity for Prompt: 7.5730
Perplexity for Prompt: 5.9766
Perplexity for Prompt: 9.0057
Perplexity for Prompt: 7.6832
Perplexity for Prompt: 5.5145
Perplexity for Prompt: 13.1189
Perplexity for Prompt: 9.1396
Perplexity for Prompt: 11.5870
Perplexity for Prompt: 6.6123
Perplexity for Prompt: 5.4388
Perplexity for Prompt: 12.4669
Perplexity for Prompt: 8.8333
Perplexity for Prompt: 12.0744
Perplexity for Prompt: 10.5712
Perplexity for Prompt: 11.6102
Perplexity for Prompt: 11.1458
Perplexity for Prompt: 9.4534
Perplexity for Prompt: 8.8805
Perplexity for Prompt: 

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

# Reference Answer
reference_answer = "Yes, the document restricts the use of Confidential Information solely for evaluation and discussion of market trends and data refinement."

# Function to compute BLEU Score
def compute_bleu(reference, hypothesis):
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()
    return sentence_bleu([reference_tokens], hypothesis_tokens)


# Function to compute ROUGE Score
def compute_rouge(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores


# Compute BLEU and ROUGE
bleu_score = compute_bleu(reference_answer, response)
rouge_scores = compute_rouge(reference_answer, response)

print(f"\nBLEU Score: {bleu_score}")
print(f"ROUGE Scores: {rouge_scores}")


BLEU Score: 0.06458538244340578
ROUGE Scores: {'rouge1': Score(precision=0.10052910052910052, recall=0.95, fmeasure=0.18181818181818182), 'rouge2': Score(precision=0.09042553191489362, recall=0.8947368421052632, fmeasure=0.1642512077294686), 'rougeL': Score(precision=0.08994708994708994, recall=0.85, fmeasure=0.16267942583732056)}


Fine Tuning